In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader

2025-02-02 15:59:09.182032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738508349.207647 3823560 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738508349.215583 3823560 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 15:59:09.243734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import sys
sys.path.append('/home/mei/nas/docker/thesis')
from model.tdpsom import TDPSOM 

In [3]:

from dataloader.ts_reader import LSTMTSDataset,collate_fn
from dataloader.pyg_reader import GraphDataset

In [4]:
latent_dim = 128
som_dim = [24, 24]
lstm_dim = 200
dropout = 0.5
learning_rate_val = 0.001
num_epochs = 5
batch_size = 16

input_channels = 163

In [5]:
data_dir = "/home/mei/nas/docker/thesis/data/hdf/train"
config = {  
    "data_dir": "/home/mei/nas/docker/thesis/data/hdf",
    "graph_dir": "/home/mei/nas/docker/thesis/data/graphs",
    "mode": "k_closest",
    "k": 3
          
}

lstm_dataset = LSTMTSDataset(data_dir,debug=True)
lstm_loader = DataLoader(lstm_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [6]:
model = TDPSOM(input_size=input_channels, latent_dim=latent_dim, som_dim=som_dim,
               learning_rate=learning_rate_val, dropout=dropout, input_channels=input_channels,
               lstm_dim=lstm_dim)

I0000 00:00:1738508376.671551 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21959 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:44:00.0, compute capability: 7.5
I0000 00:00:1738508376.673880 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21917 MB memory:  -> device: 1, name: Quadro RTX 6000, pci bus id: 0000:45:00.0, compute capability: 7.5
I0000 00:00:1738508376.675637 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22805 MB memory:  -> device: 2, name: Quadro RTX 6000, pci bus id: 0000:46:00.0, compute capability: 7.5
I0000 00:00:1738508376.677316 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 7 MB memory:  -> device: 3, name: Quadro RTX 6000, pci bus id: 0000:47:00.0, compute capability: 7.5
I0000 00:00:1738508376.678905 3823560 gpu_device.cc:2022] Created device /job:localhost/repl

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

num_epochs = 10

for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = 0
    for (flats, seqs_padded, seq_lengths), labels, ids in lstm_loader:
        # 将 PyTorch Tensor 转换为 numpy 数组
        # 注意：如果你的 tensor 在 GPU 上，则调用 .cpu().numpy()
        batch_data = seqs_padded.cpu().numpy() if seqs_padded.is_cuda else seqs_padded.numpy()
        
        feed_dict = {
            model.inputs: batch_data,
            model.is_training: True
        }
        # 执行一次训练步骤并获得当前 loss
        _, loss_val = sess.run([model.optimize, model.loss], feed_dict=feed_dict)
        epoch_loss += loss_val
        num_batches += 1
    print("Epoch {}: Average Loss = {:.4f}".format(epoch+1, epoch_loss/num_batches))
    
sess.close()


I0000 00:00:1738508383.327045 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21959 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:44:00.0, compute capability: 7.5
I0000 00:00:1738508383.328386 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21917 MB memory:  -> device: 1, name: Quadro RTX 6000, pci bus id: 0000:45:00.0, compute capability: 7.5
I0000 00:00:1738508383.329513 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22805 MB memory:  -> device: 2, name: Quadro RTX 6000, pci bus id: 0000:46:00.0, compute capability: 7.5
I0000 00:00:1738508383.331162 3823560 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 7 MB memory:  -> device: 3, name: Quadro RTX 6000, pci bus id: 0000:47:00.0, compute capability: 7.5
I0000 00:00:1738508383.332305 3823560 gpu_device.cc:2022] Created device /job:localhost/repl

Epoch 1: Average Loss = nan
Epoch 2: Average Loss = nan
Epoch 3: Average Loss = nan
Epoch 4: Average Loss = nan
Epoch 5: Average Loss = nan


KeyboardInterrupt: 

In [ ]:
# 在训练结束后，将 is_training 设为 False，对一个 batch 进行前向传播，提取 SOM 坐标以进行可视化
feed_dict = {model.inputs: data[0:batch_size], model.is_training: False}
x_recon_dist_val, z_seq_val, pred_dist_val, som_coords_val = sess.run(
    [model.reconstruction_e, tf.reshape(model.z_e_sample, [model.batch_size, model.step_size, model.latent_dim]),
     model.prediction, 
     # SOM 坐标计算部分在 forward 中已经实现
     tf.reshape(tf.stack([tf.cast(tf.div(tf.argmin(tf.reduce_sum(tf.square(tf.expand_dims(tf.reshape(model.z_e_sample, [model.batch_size, model.step_size, model.latent_dim]), 0)
                        - tf.reshape(model.embeddings, [-1, model.latent_dim])), axis=-1), model.som_dim[1]), tf.float32),
                   tf.cast(tf.mod(tf.argmin(tf.reduce_sum(tf.square(tf.expand_dims(tf.reshape(model.z_e_sample, [model.batch_size, model.step_size, model.latent_dim]), 0)
                        - tf.reshape(model.embeddings, [-1, model.latent_dim])), axis=-1), model.som_dim[1]), tf.float32)
                  ], axis=1), [model.batch_size, model.step_size, 2])
    ],
    feed_dict=feed_dict
)

# 这里直接使用模型 forward 中计算的 SOM 坐标，若 forward 返回 som_coords 则直接提取：
# 假设 som_coords_val 为 [batch_size, step_size, 2]
som_coords_val = sess.run(model.forward(data[0:batch_size])[3], feed_dict={model.is_training: False})
print("SOM 坐标形状:", som_coords_val.shape)

# 对 batch 中第一个样本的 SOM 坐标轨迹进行可视化
import matplotlib.pyplot as plt
som_traj = som_coords_val[0]  # shape: [step_size, 2]
plt.figure(figsize=(6,6))
plt.plot(som_traj[:, 0], som_traj[:, 1], marker='o', linestyle='-')
for i, (x_val, y_val) in enumerate(som_traj):
    plt.text(x_val, y_val, str(i), fontsize=8)
plt.xlabel("SOM Grid X")
plt.ylabel("SOM Grid Y")
plt.title("病人健康状态在 SOM 网格中的轨迹")
plt.grid(True)
plt.show()

sess.close()